In [ ]:
#note: must use a microsim kernel or a kernel that can load all necessary python modules

In [11]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandarallel import pandarallel
import multiprocessing as mp

In [12]:
from microsim.bp_treatment_strategies import *

from microsim.outcome_model_repository import OutcomeModelRepository
from microsim.outcome_model_type import OutcomeModelType
from microsim.person import Person

from microsim.population import NHANESDirectSamplePopulation

from microsim.bp_treatment_strategies import SprintTreatment

from microsim.sim_settings import simSettings

from microsim.trials.trial import Trial
from microsim.trials.trialset import TrialsetParallel, TrialsetSerial
from microsim.trials.trial_description import TrialDescription
from microsim.trials.logistic_regression_analysis import LogisticRegressionAnalysis
from microsim.trials.linear_regression_analysis import LinearRegressionAnalysis
from microsim.trials.outcome_assessor import OutcomeAssessor
from microsim.trials.attribute_outcome_assessor import AttributeOutcomeAssessor
from microsim.trials.attribute_outcome_assessor import AssessmentMethod
from microsim.trials.risk_filter import RiskFilter
from microsim.trials.trial_utils import get_analysis_name, randomizationSchema

In [13]:
#any microsim dir will work, just need to access the NHANES data
microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/CODE/microsim"
os.chdir(microsimDir)

In [14]:
#at the beginning turn on pandarallel if you are planning to use TrialsetSerial
pandarallel.initialize(verbose=1) #microsim by default now does not initialize pandarallel
simSettings.pandarallelFlag = True #with this flag all new population instances will be set to use pandarallel

In [15]:
#with pandarallel on these two steps are faster
pop = NHANESDirectSamplePopulation(10000, 1999)
#alive, df = pop.advance_vectorized(30)

In [16]:
anyEvent2 = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH, OutcomeType.STROKE, OutcomeType.MI, OutcomeType.DEMENTIA, OutcomeAssessor.CI]))
cogEvent = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.CI,OutcomeType.DEMENTIA]))
vascularEventOrDeath = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH, OutcomeType.STROKE, OutcomeType.MI]))
anyEvent = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH, OutcomeType.STROKE, OutcomeType.MI, OutcomeType.DEMENTIA]))
death = LogisticRegressionAnalysis(OutcomeAssessor([OutcomeAssessor.DEATH]))
qalys = LinearRegressionAnalysis(AttributeOutcomeAssessor("_qalys", AssessmentMethod.SUM))
meanGCP = LinearRegressionAnalysis(AttributeOutcomeAssessor("_gcp", AssessmentMethod.MEAN))
lastGCP = LinearRegressionAnalysis(AttributeOutcomeAssessor("_gcp", AssessmentMethod.LAST))

In [17]:
#we will later define how many processes to launch, this is just for validation
#and so that we do not set the number of processes to a number greater than the number of available cores
#print("code has access to cores ",len(os.sched_getaffinity(0)))

In [18]:
#these are usually set from the input file, but to make things easier here 
#just set them on the notebook (which means no script will test for their meaningfulness)
inputSampleSizes = [1000,2000,10000] #for quick tests
#inputSampleSizes = [100, 200, 500, 1000, 5000, 10000, 15000, 20000]
inputDurations = [1,2] #for quick tests
#inputDurations = [3,5,10,15,20]
#inputDemThresholds = [2.4845839854531493e-08, 0.00018576417292080007, 0.0012917270937081809, 0.005870510161620921, 0.025739443157677927]
#inputCvThresholds = [1.167603052003119e-06, 0.0008193743487641601, 0.0026191105926681, 0.006091251406939853, 0.0132184645579298]
#inputDemThresholds = [2.4845839854531493e-08,0.00018576417292080007] #for quick tests
#inputCvThresholds = [1.167603052003119e-06,0.0008193743487641601] #for quick tests
inputDemThresholds = [2.4845839854531493e-08] #for quick tests
inputCvThresholds = [1.167603052003119e-06] #for quick tests
inputTrialsetSize = 3
inputProcesses = 3

In [19]:
#if you try running a TrialsetParallel instance with pandarallel on, you will get an exception
#but these could be included in the TrialsetParallel.run() method
simSettings.pandarallelFlag = False #any new population instance will have pandarallel off 
pop.use_pandarallel(False) #for mp to pickle this instance that used pandarallel, must change its attributes

In [20]:
%%time
#a drawback of including these thresholds outside of the trialset instances is that these cannot be parallelized
#so in every iteration, I have to wait until all trials of the trialset are done in order to proceed to the next
#iteration/trialset (=time lost)
#the more thresholds I have, the greater the time lost 
#but it is relative to have many trials the set has and how many cores I am using
results = None
for dem in inputDemThresholds:
    for cv in inputCvThresholds:
            myRiskFilter = RiskFilter({OutcomeModelType.DEMENTIA : dem , OutcomeModelType.CARDIOVASCULAR : cv})
            #this trial description will be the same for all trials in the trial set
            desc = TrialDescription(sampleSizes= inputSampleSizes,
                                                durations = inputDurations,
                                                inclusionFilter=myRiskFilter.exceedsThresholds,
                                                exclusionFilter=None,
                                                analyses=[anyEvent2,
                                                          cogEvent,
                                                        death,
                                                        anyEvent,
                                                        vascularEventOrDeath,
                                                        qalys,
                                                        meanGCP,
                                                        lastGCP],
                                                treatment=SprintTreatment(),
                                                randomizationSchema=randomizationSchema)
            #create the set
            trialset = TrialsetParallel(desc,
                                        pop,
                                        inputTrialsetSize,
                                        inputProcesses, #an extra argument for TrialsetParallel
                                        additionalLabels=myRiskFilter._outcomeRiskThresholds)
            #trialset = TrialsetSerial(desc,
            #                          pop,
            #                          inputTrialsetSize,
            #                          additionalLabels=myRiskFilter._outcomeRiskThresholds)
            if __name__ ==  '__main__': #launching processes with multiprocesses requires this
                if results is None:
                    results = trialset.run()
                else:
                    results = pd.concat([results,trialset.run()]).reset_index(drop=True)

starting trial 0 now
starting trial 1 now
starting trial 2 now


/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS
/Users/deligkaris.1/Library/CloudStorage/OneDrive-TheOhioStateUniversityWexnerMedicalCenter/MICROSIM/CODE/microsim/microsim/population.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  totalBPMedsAddedCapped.loc[totalBPMedsAddedCapped >= BaseTreatmentStrategy.MAX_BP_MEDS] = BaseTreatmentStrategy.MAX_BP_MEDS


/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

...

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is

/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol. The list of unsupported keyword arguments passed include: method_kwargs. After release 0.14, this will raise.
  warnings.warn(
/Users/deligkaris.1/Library/Caches/pypoetry/virtualenvs/microsim-54NoInYP-py3.9/lib/python3.9/site-packages/statsmodels/base/optimizer.py:17: FutureWarning:

ending trial 0 now
ending trial 1 now
ending trial 2 now
CPU times: user 315 ms, sys: 57.7 ms, total: 372 ms
Wall time: 1min 37s


In [21]:
results.head()

,reg,se,pvalue,intercept,meanUntreated,meanTreated,duration,sampleSize,outcome,analysis,OutcomeModelType.DEMENTIA,OutcomeModelType.CARDIOVASCULAR
0,0.041189,0.040140,0.304833,0.144754,0.536125,0.546352,1,10000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001
1,0.040361,0.040138,0.314632,0.143956,0.535927,0.545949,1,10000,ci-dementia-,logisticRegression-ci-dementia-,2.484584e-08,0.000001
2,0.526438,0.730571,0.471165,-7.425557,0.000595,0.001008,1,10000,death-,logisticRegression-death-,2.484584e-08,0.000001
3,0.015216,0.632772,0.980816,-6.914334,0.000992,0.001008,1,10000,death-stroke-mi-dementia-,logisticRegression-death-stroke-mi-dementia-,2.484584e-08,0.000001
4,0.015216,0.632772,0.980816,-6.914334,0.000992,0.001008,1,10000,death-stroke-mi-,logisticRegression-death-stroke-mi-,2.484584e-08,0.000001


In [86]:
results.head(50)

,reg,se,pvalue,intercept,meanUntreated,meanTreated,duration,sampleSize,outcome,analysis,OutcomeModelType.DEMENTIA,OutcomeModelType.CARDIOVASCULAR,meanDifference,testLinear,diff,oddsControl,oddsTreated,absRiskControl,absRiskTreated
0,4.118899e-02,4.014034e-02,3.048329e-01,0.144754,0.536125,0.546352,1,10000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001,0.010227,False,3.096216e-02,1.155755e+00,1.204354e+00,5.361254e-01,0.546352
1,4.036099e-02,4.013825e-02,3.146324e-01,0.143956,0.535927,0.545949,1,10000,ci-dementia-,logisticRegression-ci-dementia-,2.484584e-08,0.000001,0.010022,False,3.033873e-02,1.154833e+00,1.202397e+00,5.359270e-01,0.545949
2,5.264384e-01,7.305708e-01,4.711648e-01,-7.425557,0.000595,0.001008,1,10000,death-,logisticRegression-death-,2.484584e-08,0.000001,0.000412,False,5.260262e-01,5.958292e-04,1.008675e-03,5.954744e-04,0.001008
3,1.521551e-02,6.327720e-01,9.808161e-01,-6.914334,0.000992,0.001008,1,10000,death-stroke-mi-dementia-,logisticRegression-death-stroke-mi-dementia-,2.484584e-08,0.000001,0.000015,False,1.520031e-02,9.934433e-04,1.008675e-03,9.924573e-04,0.001008
4,1.521551e-02,6.327720e-01,9.808161e-01,-6.914334,0.000992,0.001008,1,10000,death-stroke-mi-,logisticRegression-death-stroke-mi-,2.484584e-08,0.000001,0.000015,False,1.520031e-02,9.934433e-04,1.008675e-03,9.924573e-04,0.001008
5,1.924111e-04,4.471553e-04,6.669864e-01,0.999405,0.999405,0.999597,1,10000,_qalys-sum,linearRegression-_qalys-sum,2.484584e-08,0.000001,0.000192,True,8.581840e-15,2.716664e+00,2.717186e+00,7.309415e-01,0.730979
6,-6.735917e-02,7.035068e-02,3.383496e-01,53.799524,53.799524,53.732164,1,10000,_gcp-mean,linearRegression-_gcp-mean,2.484584e-08,0.000001,-0.067359,True,6.949719e-13,2.316521e+23,2.165621e+23,1.000000e+00,1.000000
7,-1.372868e-01,1.402368e-01,3.276213e-01,50.878722,50.878722,50.741435,1,10000,_gcp-last,linearRegression-_gcp-last,2.484584e-08,0.000001,-0.137287,True,6.039613e-13,1.248384e+22,1.088241e+22,1.000000e+00,1.000000
8,8.143047e-02,1.276355e-01,5.234788e-01,0.224944,0.556000,0.576000,1,1000,death-stroke-mi-dementia-ci-,logisticRegression-death-stroke-mi-dementia-ci-,2.484584e-08,0.000001,0.020000,False,6.143047e-02,1.252252e+00,1.358491e+00,5.560000e-01,0.576000
9,8.143047e-02,1.276355e-01,5.234788e-01,0.224944,0.556000,0.576000,1,1000,ci-dementia-,logisticRegression-ci-dementia-,2.484584e-08,0.000001,0.020000,False,6.143047e-02,1.252252e+00,1.358491e+00,5.560000e-01,0.576000


In [26]:
results["meanDifference"] = results["meanTreated"] - results["meanUntreated"]
results["diffLinear"] = results["reg"] - results["meanDifference"]
results["testLinear"] = (abs(results["diff"]) < 10**(-10))

In [83]:
#check the linear regression results (ignore logistic regression results right now)
results.loc[ (results["outcome"]=="_qalys-sum") | 
             (results["outcome"]=="_gcp-mean") |
            (results["outcome"]=="_gcp-last"), ["testLinear", "reg" ,"meanDifference","diff"] ]

,testLinear,reg,meanDifference,diff
5,True,1.924111e-04,0.000192,8.581840e-15
6,True,-6.735917e-02,-0.067359,6.949719e-13
7,True,-1.372868e-01,-0.137287,6.039613e-13
13,True,4.163336e-16,0.000000,4.163336e-16
14,True,6.800000e-02,0.068000,2.502165e-14
15,True,5.800000e-02,0.058000,1.706968e-14
21,True,4.163336e-16,0.000000,4.163336e-16
22,True,8.600000e-02,0.086000,3.222422e-14
23,True,2.320000e-01,0.232000,3.469447e-14
29,True,4.163336e-16,0.000000,4.163336e-16


In [85]:
import numpy as np
results["oddsControl"] = np.exp(results["intercept"])
results["oddsTreated"] = np.exp(results["intercept"]+results["reg"])
results["absRiskControl"] = results["oddsControl"] / (1+results["oddsControl"])
results["absRiskTreated"] = results["oddsTreated"] / (1+results["oddsTreated"])

In [93]:
results["diffControl"]= abs(results["absRiskControl"] - results["meanUntreated"]) 
results["diffTreated"]= abs(results["absRiskTreated"] - results["meanTreated"])
results["testLogisticControl"] = (abs(results["diffControl"]) < 10**(-6))
results["testLogisticTreated"] = (abs(results["diffTreated"]) < 10**(-6))

In [94]:
#check the logistic regression results (ignore linear regression results right now)
results.loc[ (results["outcome"]=="death-stroke-mi-dementia-ci-") | 
             (results["outcome"]=="ci-dementia") |
             (results["outcome"]=="death") |
             (results["outcome"]=="death-stroke-mi-"), 
              ["testLogisticTreated","testLogisticControl","diffControl", "diffTreated"] ]

,testLogisticTreated,testLogisticControl,diffControl,diffTreated
0,True,True,2.220446e-16,2.220446e-16
4,True,True,1.604619e-17,1.452831e-17
8,True,True,1.110223e-16,1.110223e-16
12,False,False,NaN,NaN
16,True,True,1.110223e-16,0.000000e+00
20,True,True,9.284003e-10,2.770763e-10
24,True,True,1.110223e-16,1.110223e-16
28,True,True,9.283959e-10,2.770756e-10
32,True,True,1.110223e-16,0.000000e+00
36,False,False,NaN,NaN


In [23]:
#test a single trial, and its regression results

In [ ]:
testTrial=Trial(desc,pop, #an extra argument for TrialsetParallel
              additionalLabels=myRiskFilter._outcomeRiskThresholds)

In [ ]:
testTrial.run()

In [ ]:
data=death.get_dataframe(testTrial.treatedPop._people.tolist(),testTrial.untreatedPop._people.tolist())

In [ ]:
import statsmodels.formula.api as smf
reg = smf.logit("outcome ~ treatment", data).fit(disp=False, method_kwargs={"warn_convergence": False})

In [ ]:
reg.params

In [ ]:
reg.params["Intercept"]

In [ ]:
reg.summary()